# Assignment 5

In [1]:
import numpy as np
import pandas as pd 
import scipy as sci
import math 
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from itertools import chain
from numpy import pi
from scipy import interpolate
from scipy.signal import freqz, welch, filtfilt, ellip
from scipy.interpolate import interp1d

from matplotlib.dates import DateFormatter, MinuteLocator, HourLocator, MonthLocator
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
from matplotlib.ticker import FixedFormatter

from textwrap import wrap

%matplotlib inline
%config InlineBackend.figure_format = 'pdf'

### Import data

In [2]:
AS = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/AllStations_temperature_h_2017.dat', 
                 sep='\s+', header=[0, 1])

DC = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/DeepCove_temperature.dat', 
                 header=2)

JB = pd.read_csv('/Users/Kev/Documents/Uvic/Python/PHYS 411 - Time Series Analysis/Data Sets/JamesBay_temperature.dat', 
                 header=2)

### Clean up data

In [3]:
# Clean up minute resolution data 
# -------------------------------

def DateInsert(Data):
    date = pd.date_range(start='2011-12-31 17:00:00.000000', 
                     freq='min', periods = len(Data))
    # Insert dates into D2 dataframe
    Data.insert(loc=0, column='Time', value=date)
    # Rename the columns
    D1 = Data.rename(index=str, columns={Data.columns[1]: "Temperature"})
    # Set index
    DM = D1.set_index('Time')
    return DM

DataM = [DC, JB]
DataMLabels = ['Deep Cove', 'James Bay']

# Insert dates
DataMIns = [DateInsert(DataM[i]) for i in range(0, len(DataM))]

In [4]:
# Clean up hour resolution data
# -----------------------------

# Convert times from MatLab time to Python Time
AS['Time'] = AS['NaN', 'NaN'].apply(lambda matlab_datenum: 
                             dt.datetime.fromordinal(int(matlab_datenum)) 
                             + dt.timedelta(days=matlab_datenum%1)
                             - dt.timedelta(days = 366)) 

# Rename the columns
AS2 = AS.rename(index=str, columns={"NaN": "MatLab Time"})

# Set time as index column
AS3 = AS2.set_index('Time')

# Get the longitudes correct 
for i in range(1, np.shape(AS3)[1]):
    New_name = float(list(AS3)[i][1]) - 360
    AS3 = AS3.rename(columns={list(AS)[i][1]: str(format(New_name, '.4f'))})

AS4 = AS3.drop('MatLab Time', axis=1, level=1)
    
AS = AS4['48.4623']['-123.3090']

### Select data time

In [30]:
# Select dates
DCs = DataMIns[0].loc['2015-06-01 00:00':'2015-09-30 23:59']['Temperature']
JBs = DataMIns[1].loc['2015-06-01 00:00':'2015-09-30 23:59']['Temperature']
DCw = DataMIns[0].loc['2014-11-01 00:00':'2015-02-28 23:59']['Temperature']
JBw = DataMIns[1].loc['2014-11-01 00:00':'2015-02-28 23:59']['Temperature']

US = AS.loc['2016-01-01 00:00':'2016-12-30 23:59']
USm = AS.loc['2016-03-01 00:00':'2016-3-31 23:59']

## Question 1: Power within a frequency band 

In [6]:
# Plot out the time series 
# ========================

fig, (DCs_Plot, JBs_Plot, DCw_Plot, JBw_Plot) = plt.subplots(4, 1, figsize=(12, 16))

DCs_Plot.plot(DCs, linewidth=1)
DCw_Plot.plot(DCw, linewidth=1)
JBs_Plot.plot(JBs, linewidth=1)
JBw_Plot.plot(JBw, linewidth=1)

DCs_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
DCw_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBs_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
JBw_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))

DCs_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
DCw_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBs_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
JBw_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))

DCs_Plot.set_title("Minute resolution measuerments of temperature at Deep Cove from 2015-06-01 to 2015-09-30")
DCw_Plot.set_title("Minute resolution measuerments of temperature at Deep Cove from 2014-11-01 to 2015-02-28")
JBs_Plot.set_title("Minute resolution measuerments of temperature at James Bay from 2015-06-01 to 2015-09-30")
JBw_Plot.set_title("Minute resolution measuerments of temperature at James Bay from 2014-11-01 to 2015-02-28")

DCs_Plot.set_xlabel("Time")
DCw_Plot.set_xlabel("Time")
JBs_Plot.set_xlabel("Time")
JBw_Plot.set_xlabel("Time")

DCs_Plot.set_ylabel(r"Temperature [$^\circ$ C]")
DCw_Plot.set_ylabel(r"Temperature [$^\circ$ C]")
JBs_Plot.set_ylabel(r"Temperature [$^\circ$ C]")
JBw_Plot.set_ylabel(r"Temperature [$^\circ$ C]")

fig.tight_layout()
plt.show()

<Figure size 864x1152 with 4 Axes>

In [7]:
# Calculate the power spectra
# ===========================

# De-Nan
DCs_DN = DCs[~np.isnan(DCs)]
JBs_DN = DCw[~np.isnan(DCw)]
DCw_DN = JBs[~np.isnan(JBs)]
JBw_DN = JBw[~np.isnan(JBw)]
FSm = 1/60

# Lengths 
[DCs_DN_L, JBs_DN_L, DCw_DN_L, JBw_DN_L] = np.array([len(DCs_DN), len(JBs_DN), len(DCw_DN), len(JBw_DN)]) / (2**2)

# Now calculate
FDCs, GDCs = welch(DCs_DN, FSm, nperseg=DCs_DN_L,\
            window=sci.signal.windows.hann(int(DCs_DN_L)),\
            noverlap = DCs_DN_L/2, nfft = DCs_DN_L, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

FDCw, GDCw = welch(DCw_DN, FSm, nperseg=JBs_DN_L,\
            window=sci.signal.windows.hann(int(JBs_DN_L)),\
            noverlap = JBs_DN_L/2, nfft = JBs_DN_L, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

FJBs, GJBs = welch(JBs_DN, FSm, nperseg=DCw_DN_L,\
            window=sci.signal.windows.hann(int(DCw_DN_L)),\
            noverlap = DCw_DN_L/2, nfft = DCw_DN_L, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

FJBw, GJBw = welch(JBw_DN, FSm, nperseg=JBw_DN_L,\
            window=sci.signal.windows.hann(int(JBw_DN_L)),\
            noverlap = JBw_DN_L/2, nfft = JBw_DN_L, detrend=False,\
            return_onesided=True, scaling = 'spectrum')

In [8]:
# Plot the power spectra
# ======================
fig, (DCs_PS, JBs_PS, DCw_PS, JBw_PS) = plt.subplots(4, 1, figsize=(12, 16))
# plt.loglog(Fx/(2*pi), Gx, label = r'PDS of $x(t)$')
DCs_PS.loglog(FDCs/(2*pi), GDCs, label = r'PDS of $x_f(t)$', linewidth=1)
JBs_PS.loglog(FJBs/(2*pi), GJBs, label = r'PDS of $x_f(t)$', linewidth=1)
DCw_PS.loglog(FDCw/(2*pi), GDCw, label = r'PDS of $x_f(t)$', linewidth=1)
JBw_PS.loglog(FJBw/(2*pi), GJBw, label = r'PDS of $x_f(t)$', linewidth=1)

DCs_PS.set_title("Power spectrum of minute resolution measuerments of temperature at Deep Cove from 2015-06-01 to 2015-09-30")
DCw_PS.set_title("Power spectrum of minute resolution measuerments of temperature at Deep Cove from 2014-11-01 to 2015-02-28")
JBs_PS.set_title("Power spectrum of minute resolution measuerments of temperature at James Bay from 2015-06-01 to 2015-09-30")
JBw_PS.set_title("Power spectrum of minute resolution measuerments of temperature at James Bay from 2014-11-01 to 2015-02-28")

DCs_PS.set_xlabel("Frequency [Hz]")
DCw_PS.set_xlabel("Frequency [Hz]")
JBs_PS.set_xlabel("Frequency [Hz]")
JBw_PS.set_xlabel("Frequency [Hz]")

DCs_PS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
DCw_PS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
JBs_PS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')
JBw_PS.set_ylabel(r'$G_{xx}(f)$ [$\frac{V^2}{Hz}$]')

fig.tight_layout()
plt.show()

<Figure size 864x1152 with 4 Axes>

#### a) Elliptic filters

In [9]:
# Filter functions
# =================

# High/Low-pass filters 
def Ellip(Cut, rp, rs, fs, order, btype):
    fn = fs/2
    Ws = Cut / fn
    b, a = ellip(order, rp, rs, Ws, btype, output='ba')
    return b, a

def EllipF(x, Cut, rp, rs, fs, order, btype):
    b, a = Ellip(CutL, rp, rs, fs, order, btype)
    y = filtfilt(b, a, x)
    return y

def GraphEllip(Cut, rp, rs, fs, order, btype, worN=2*10**5):
    b, a = Ellip(Cut, rp, rs, fs, order, btype)
    w, h = freqz(b, a, worN)
    return b, a, w, h

# Bandpass filters 
def EllipBP(CutL, CutH, rp, rs, fs, order, btype):
    fn = fs/2
    low = CutL / fn
    high = CutH / fn
    b, a = ellip(order, rp, rs, [low, high], btype, output='ba')
    return b, a

def EllipFBP(x, CutL, CutH, rp, rs, fs, order, btype):
    b, a = EllipBP(CutL, CutH, rp, rs, fs, order, btype)
    y = filtfilt(b, a, x)
    return y

def GraphEllipBP(CutL, CutH, rp, rs, fs, order, btype, worN=2*10**5):
    b, a = EllipBP(CutL, CutH, rp, rs, fs, order, btype)
    w, h = freqz(b, a, worN)
    return b, a, w, h

In [10]:
# Ellipse filter settings
[CutH, rpH, rsH, orderH, btypeH] = [0.82*10**(-5)/(2*pi), 25, 150, 2, 'highpass']
[CutL, rpL, rsL, orderL, btypeL] = [1.63*10**(-5)/(2*pi), 25, 150, 2, 'lowpass']
[CutBL, CutBH, rpB, rsB, orderB, btypeB] = [0.825*10**(-5)/(2*pi), 1.63*10**(-5)/(2*pi), 25, 150, 2, 'bandpass']

In [11]:
# Plot the filter
fig, (ellip_plot) = plt.subplots(1, 1, figsize=(10, 4))

bH, aH, wH, hH = GraphEllip(Cut=CutH, rp=rpH, rs=rsH, fs=FSm, order=orderH, btype=btypeH)
bL, aL, wL, hL = GraphEllip(Cut=CutL, rp=rpL, rs=rsL, fs=FSm, order=orderL, btype=btypeL)
bB, aB, wB, hB = GraphEllipBP(CutL=CutBL, CutH=CutBH, rp=rpB, rs=rsB, fs=FSm, order=orderB, btype=btypeB)

ellip_plot.loglog(FDCs/(2*pi), GDCs, label = r'PDS of $x_f(t)$', linewidth=1)
ellip_plot.loglog(FSm/(2*pi) * wH, abs(hH), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz".format(btypeH, orderH, CutH))
ellip_plot.loglog(FSm/(2*pi) * wL, abs(hL), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz".format(btypeL, orderL, CutL))
ellip_plot.loglog(FSm/(2*pi) * wB, abs(hB), label="Type: {0}, Order: {1}, Cut-off: {2:.1e} Hz and {3:.1e} Hz".format(btypeB, orderB, CutBL, CutBH))
# f3d1.plot([0, fs3/2], [sqrt(0.5), sqrt(0.5)], '--', label=r'$\sqrt{0.5}$')
ellip_plot.set_title('Elliptic filter frequency responce')
ellip_plot.set_xlabel(r'Frequency ($Hz$)')
ellip_plot.set_ylabel(r'$H(\omega)$')
ellip_plot.grid(True)
ellip_plot.legend(loc=3)
# ellip_plot.set_xlim(10**(-6), 10**(-5))

# Plot filtered x(t)
# f3d2.plot(t3, x1, label=r'$x_1(t)$')
# f3d2.plot(t3, f1, label=r'$x_f(t)$')
# title3b = r'Isolation of $x1=\cos(22\pi t)$ signal from x(t) via a Butterworth filter with cut-off frequencies at {0} $rad/s$ and {1} $rad/s$'.format(int(CutL), int(CutH))
# f3d2.set_title("\n".join(wrap(title3b, 90)))
# f3d2.set_xlabel(r'Time ($s$)')
# f3d2.set_ylabel(r'$x_f(t)$')
# f3d2.legend(loc=1)
# f3d2.set_xlim(-1, 1)

fig.tight_layout()
plt.show()

<Figure size 720x288 with 1 Axes>

In [13]:
# Filter the time series 
DCsFH = EllipF(DCs, Cut=CutH, rp=rpH, rs=rsH, fs=FSm, order=orderH, btype=btypeH)
DCsFL = EllipF(DCs, Cut=CutL, rp=rpL, rs=rsL, fs=FSm, order=orderL, btype=btypeL)
DCsFHL = EllipF(DCsFH, Cut=CutL, rp=rpL, rs=rsL, fs=FSm, order=orderL, btype=btypeL)
DCsFB = EllipFBP(DCs, CutL=CutBL, CutH=CutBH, rp=rpB, rs=rsB, fs=FSm, order=orderB, btype=btypeB)

# i_DCs = DCs.index.values
dr_DCs = pd.date_range(DCs.index[0], DCs.index[-1], freq='m')
t_DCs = DCs.index.values

plt.figure(figsize=(10, 4))
plt.plot(t_DCs, DCsFH)

plt.figure(figsize=(10, 4))
plt.plot(t_DCs, DCsFL)

plt.figure(figsize=(10, 4))
plt.plot(t_DCs, DCsFHL)

plt.figure(figsize=(10, 4))
plt.plot(t_DCs, DCsFB)

/Users/Kev/anaconda3/lib/python3.7/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


<Figure size 720x288 with 1 Axes>

<Figure size 720x288 with 1 Axes>

<Figure size 720x288 with 1 Axes>

<Figure size 720x288 with 1 Axes>

## Question 2: Spatial interpolation 

## Question 3: Interpolation

In [31]:
fig, (US_Plot, USm_Plot) = plt.subplots(2, 1, figsize=(12, 8))

US_Plot.plot(US, linewidth=1)
US_Plot.xaxis.set_major_locator(mp.dates.MonthLocator(bymonthday = (1)))
US_Plot.xaxis.set_major_formatter(DateFormatter('%b-%Y'))
US_Plot.set_title("Hour resolution measuerments of temperature at UVicSCI from 2015-01-01 to 2015-12-31")
US_Plot.set_xlabel("Time")
US_Plot.set_ylabel(r"Temperature [$^\circ$ C]")

USm_Plot.plot(USm, linewidth=1)
USm_Plot.xaxis.set_major_formatter(DateFormatter('%d-%b'))
USm_Plot.set_title("Hour resolution measuerments of temperature at UVicSCI from 2015-03-01 to 2015-03-31")
USm_Plot.set_xlabel("Time")
USm_Plot.set_ylabel(r"Temperature [$^\circ$ C]")
plt.xticks(rotation=90)
fig.tight_layout()
plt.show()

<Figure size 864x576 with 2 Axes>

In [75]:
# Interpolate the data
# ====================

# De-Nan
# De-Nan
USm_DN = USm[~np.isnan(USm)]
x = np.linspace(0, len(USm), len(USm))

t_USm = USm.index.values
USm_lin = interp1d(x, np.array(USm), kind='linear')
# USm_lin = interp1d(t_USm, np.array(USm), kind='cubic')

In [82]:
plt.figure(figsize=(10,4))
plt.plot(x, np.array(USm), linewidth=1)
plt.plot(np.linspace(0, len(USm), len(USm)*10), USm_lin(np.linspace(0, len(USm), len(USm)*10)))

<Figure size 720x288 with 1 Axes>

In [83]:
USm_lin(np.linspace(0, len(USm), len(USm)*0.1))

/Users/Kev/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  """Entry point for launching an IPython kernel.


array([ 8.2       ,  9.11835616,  7.66054795,  9.04753425,  9.86178082,
        8.70479452, 11.76780822,  9.72547945, 10.91493151,  8.99821918,
        9.04287671, 13.51945205, 10.50945205, 11.54739726,  7.7639726 ,
        6.86452055,  7.97589041,  5.54835616,  9.47410959,  6.79739726,
        8.28150685,  8.49945205,  9.37986301,  8.76068493,  7.25616438,
        9.07808219,  8.20178082,  9.79506849,  6.77383562,  5.90876712,
        6.10109589,  6.24273973,  7.8160274 ,  5.50753425,  6.16246575,
        6.98534247,  5.12465753,  9.18493151,  5.52315068,  8.92109589,
        7.72534247,  9.4230137 , 10.38219178,  8.49328767, 12.54835616,
       10.17410959,  9.85630137,  9.22191781,  8.38369863, 10.26      ,
        7.58986301, 10.73547945,  9.09219178,  8.68794521,  8.34945205,
        7.56041096,  8.39958904,  6.10780822, 10.50068493,  7.28      ,
        8.99520548,  9.40917808,  7.74506849,  8.49027397,  5.91136986,
       10.0439726 ,  8.29013699, 10.88369863, 13.6269863 ,  9.05

In [74]:
len(x), len(USm)

(744, 744)